In [1]:
import numpy as np # for array, linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt # to crreate stat and visualize data
import matplotlib as mpl  # to visualize data
import seaborn as sns # to visualize data
import matplotlib.colors as mcolors # to visualize colors
import string # collection of alphabets, words or other characters
import re # regular expression support
import nltk 
from nltk.corpus import stopwords
from string import punctuation 
# import gensim # representing documents as semantic vectors


from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/I570078/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/I570078/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
pd.set_option('display.max_colwidth', None)

## Sentiwordnet with Text-Pre-Edditing

In [3]:
df_sentiwordnet_without_stopwords = pd.read_csv('../../Data-Preperation/traindata_without_stopwords_preprocessed.csv')

### Satzteile und Stopwords entfernen, die keinen Sentiment-Score aufweisen

In [4]:
df_sentiwordnet_without_stopwords

,target,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
...,...,...
1599995,4,Just woke up. Having no school is the best feeling ever
1599996,4,TheWDB.com - Very cool to hear old Walt interviews! ♫ http://blip.fm/~8bmta
1599997,4,Are you ready for your MoJo Makeover? Ask me for details
1599998,4,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur


In [11]:
df_sentiwordnet_without_stopwords['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [12]:
def naiveSentiment(review):
    reviewPolarity = 0.0
    numExceptions = 0
    words = review.lower().split()
    
    for word in words:
        numMeanings = 0
        weight = 0.0
        synsets = list(swn.senti_synsets(word))
        if synsets:
            for meaning in synsets:
                pos_score = meaning.pos_score()
                neg_score = meaning.neg_score()
                if pos_score > neg_score:
                    weight += (pos_score - neg_score)
                    numMeanings += 1
                elif pos_score < neg_score:
                    weight -= (neg_score - pos_score)
                    numMeanings += 1
        else:
            numExceptions += 1
        if numMeanings > 0:
            reviewPolarity += (weight / numMeanings)
    return reviewPolarity

In [13]:
df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'] = df_sentiwordnet_without_stopwords['Text'].apply(lambda x: 'positive' if naiveSentiment(x) >= 0 else 'negative')


In [15]:
df_sentiwordnet_without_stopwords

,target,Text,Ground_Truth_Label,Sentiwordnet_Classification
0,0,Awww bummer shoulda got David Carr Third Day,negative,negative
1,0,upset update Facebook texting might cry result School today also Blah,negative,negative
2,0,dived many times ball Managed save 50 rest go bounds,negative,positive
3,0,whole body feels itchy like fire,negative,negative
4,0,behaving mad see,negative,positive
...,...,...,...,...
1599995,4,woke school best feeling ever,positive,positive
1599996,4,TheWDB com cool hear old Walt interviews ♫,positive,positive
1599997,4,ready MoJo Makeover Ask details,positive,positive
1599998,4,Happy 38th Birthday boo alll time Tupac Amaru Shakur,positive,positive


In [17]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_sentiwordnet_without_stopwords['Ground_Truth_Label'], df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'])
print(f'Accuracy: {accuracy}')

Accuracy: 0.608200625
